In [1]:
%load_ext autoreload
%autoreload 2
import netCDF4
import xarray as xr
import numpy as np
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
from hmpai.pytorch.training import train, validate, calculate_class_weights, train_and_test, k_fold_cross_validate, test
from hmpai.pytorch.generators import SAT1Dataset
from hmpai.data import SAT1_STAGES_ACCURACY, SAT_CLASSES_ACCURACY
from hmpai.visualization import plot_confusion_matrix
from hmpai.normalization import *
from torchinfo import summary
from torchvision.transforms import Compose
from hmpai.pytorch.transforms import *
from hmpai.utilities import print_results, CHANNELS_2D, AR_SAT1_CHANNELS
from torch.utils.data import DataLoader
from mne.io import read_info
from pathlib import Path
from hmpai.pytorch.models import *
from hmpai.training import split_data_on_participants
import pandas as pd
from hmpai.pytorch.utilities import DEVICE, set_global_seed, get_summary_str, save_model, load_model

import os
DATA_PATH = Path(os.getenv("DATA_PATH"))

In [8]:
set_global_seed(42)
data_path_sat2 = DATA_PATH / "sat2/stage_data_proba_100hz.nc"
dataset_sat2 = xr.open_dataset(data_path_sat2)
# data_path_sat1 = DATA_PATH / "sat1/stage_data_100hz.nc"
# dataset_sat1 = xr.load_dataset(data_path_sat1)
# dataset_sat1 = dataset_sat1.rename_vars({'RT': 'rt'})

dataset_sat2 = dataset_sat2.where(dataset_sat2.event_name.str.contains("accuracy"), drop=True)
# dataset_sp = dataset_sat1.where(dataset_sat1.event_name.str.contains("speed"), drop=True)

In [3]:
# shape_topological = False
# info_to_keep = ['rt']
# train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
#     dataset_sat2, 60, norm_min1_to_1
# )
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
# train_dataset = SAT1Dataset(train_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# val_dataset = SAT1Dataset(val_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)
# test_dataset = SAT1Dataset(test_data, shape_topological=shape_topological, labels=SAT_CLASSES_ACCURACY, set_to_zero=True, info_to_keep=info_to_keep, order_by_rt=False)

In [9]:
shape_topological = False
info_to_keep = ["rt"]
train_data_sat2, val_data_sat2, test_data_sat2 = split_data_on_participants(
    dataset_sat2, 90, norm_min1_to_1
)
# train_data_sat2 = train_data_sat2.where(train_data_sat2.labels != 0, drop=True)
# val_data_sat2 = val_data_sat2.where(val_data_sat2.labels != 0, drop=True)
# test_data_sat2 = test_data_sat2.where(test_data_sat2.labels != 0, drop=True)
# train_data_sat1, val_data_sat1, test_data_sat1 = split_data_on_participants(
#     dataset_sat1, 60, norm_min1_to_1
# )
# train_data = xr.merge([train_data_sat2, train_data_sat1])
# val_data = xr.merge([val_data_sat2, val_data_sat1])
# test_data = xr.merge([test_data_sat2, test_data_sat1])
train_dataset = SAT1Dataset(
    train_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    transform=Compose([TimeMasking(0.5, 20), ChannelsDropout(0.5, 0.4)]),
)
val_dataset = SAT1Dataset(
    val_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)
test_dataset = SAT1Dataset(
    test_data_sat2,
    shape_topological=shape_topological,
    labels=SAT_CLASSES_ACCURACY,
    set_to_zero=True,
    info_to_keep=info_to_keep,
    order_by_rt=False,
    # transform=Compose([RandomCropTransform()]),
)

['S12', 'S1']


In [7]:
# Train model
model = MambaModel(embed_dim=256, n_channels=30, n_classes=len(SAT_CLASSES_ACCURACY), n_layers=6, global_pool=False, dropout=0.1)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=0,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=100,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.01,
    lr=0.0001
)

  0%|          | 0/249 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    114437
           1       0.71      0.94      0.81     10275
           2       0.86      0.68      0.76     31900
           3       0.77      0.11      0.19     30079
           4       0.13      0.91      0.22      4961

    accuracy                           0.79    191652
   macro avg       0.69      0.73      0.59    191652
weighted avg       0.90      0.79      0.80    191652



  0%|          | 0/249 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    116597
           1       0.67      0.96      0.79     10275
           2       0.82      0.70      0.76     31900
           3       0.77      0.17      0.27     30079
           4       0.14      0.84      0.23      4961

    accuracy                           0.81    193812
   macro avg       0.68      0.73      0.61    193812
weighted avg       0.90      0.81      0.81    193812



  0%|          | 0/249 [00:00<?, ? batch/s]

...

  0%|          | 0/249 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    107653
           1       0.65      0.96      0.77     10275
           2       0.72      0.77      0.75     31900
           3       0.79      0.21      0.33     30079
           4       0.16      0.69      0.26      4961

    accuracy                           0.81    184868
   macro avg       0.66      0.72      0.62    184868
weighted avg       0.88      0.81      0.81    184868



  0%|          | 0/249 [00:00<?, ? batch/s]

              precision    recall  f1-score   support

           0       1.00      0.98      0.99    114449
           1       0.69      0.95      0.80     10275
           2       0.74      0.80      0.77     31900
           3       0.80      0.22      0.34     30079
           4       0.15      0.69      0.25      4961

    accuracy                           0.82    191664
   macro avg       0.68      0.73      0.63    191664
weighted avg       0.89      0.82      0.82    191664



[{'0.0': {'precision': 1.0,
   'recall': 0.9746244815603632,
   'f1-score': 0.9871491928330672,
   'support': 71368.0},
  '1.0': {'precision': 0.701750625223294,
   'recall': 0.9254617414248021,
   'f1-score': 0.7982281464623887,
   'support': 10612.0},
  '2.0': {'precision': 0.6477193794859921,
   'recall': 0.86352587104989,
   'f1-score': 0.7402139939473118,
   'support': 25917.0},
  '3.0': {'precision': 0.7756073739470151,
   'recall': 0.24868864010021138,
   'f1-score': 0.3766191421881021,
   'support': 25546.0},
  '4.0': {'precision': 0.24347605776539144,
   'recall': 0.6143191988067335,
   'f1-score': 0.3487359380670134,
   'support': 4693.0},
  'accuracy': 0.8035124804540453,
  'macro avg': {'precision': 0.6737106872843386,
   'recall': 0.7253239865884,
   'f1-score': 0.6501892826995767,
   'support': 138136.0},
  'weighted avg': {'precision': 0.8437932176040387,
   'recall': 0.8035124804540453,
   'f1-score': 0.7917090195835078,
   'support': 138136.0}}]

In [18]:
model = Seq2SeqTransformer(d_model=30, ff_dim=1024, num_heads=8, num_layers=6, num_classes=len(SAT_CLASSES_ACCURACY), emb_dim=512)
model.set_pretraining(False)
train_and_test(
    model,
    train_dataset,
    test_dataset,
    val_dataset,
    logs_path=Path("../logs/"),
    workers=4,
    batch_size=64,
    labels=SAT_CLASSES_ACCURACY,
    epochs=10,
    # epochs=1,
    weight_decay=0.001,
    label_smoothing=0.1,
    lr=0.00001
)

/opt/conda/lib/python3.10/site-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


  0%|          | 0/162 [00:00<?, ? batch/s]

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.01      1.00      0.02      7550
           1       0.00      0.00      0.00     37300
           2       0.00      0.00      0.00    109161
           3       0.00      0.00      0.00    106184
           4       0.00      0.00      0.00     17640
           5       0.00      0.00      0.00    401209

    accuracy                           0.01    679044
   macro avg       0.00      0.17      0.00    679044
weighted avg       0.00      0.01      0.00    679044



  0%|          | 0/162 [00:00<?, ? batch/s]

...

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


              precision    recall  f1-score   support

           0       0.01      1.00      0.02      7550
           1       0.00      0.00      0.00     37300
           2       0.00      0.00      0.00    109161
           3       0.00      0.00      0.00    106184
           4       0.00      0.00      0.00     17640
           5       0.00      0.00      0.00    402029

    accuracy                           0.01    679864
   macro avg       0.00      0.17      0.00    679864
weighted avg       0.00      0.01      0.00    679864



/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


  0%|          | 0/162 [00:00<?, ? batch/s]

KeyboardInterrupt: 